In [ ]:
# Mout Google Drive
# https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# %pwd %ls
# run github settings
%run /content/drive/MyDrive/CNNStanford/pytorch/pytorch_sandbox/Colab_Helper.ipynb

In [ ]:
MESSAGE = "clean file & gitignore again"
!git config --global user.email "ronyginosar@mail.huji.ac.il"
!git config --global user.name "ronyginosar"
!git add .

In [ ]:
!git commit -m "{MESSAGE}"
!git push "{GIT_PATH}"

In [ ]:
import torch
import sys
import os
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from exercises.part5_train.ex1_train_cifar import tensor_show
from exercises.part3_nn_modules.ex1 import Ex1Net
import torchvision

batch_size = 4
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def main(argv):
    """
    Write your evaluation code here ...
    """
    #Torchmetrics ??
    # eval vs validation??
    # skeleton see part 6
    CHECKPOINT_PATH = os.path.join(os.getcwd(), "checkpoints", "latest.pt")

    model = Ex1Net(in_channels=[3, 32, 64, 128],
                 out_channels=[32, 64, 128, 256],
                 pools=['max', 'max', 'max', 'avg'],
                 num_classes=10)
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['model'])
    # must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference
    model.eval()
    # print(model.state_dict())


    normalize_transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    root_path = os.path.join(os.getcwd(), "data")  # if needed: "exercises", "part5_train"
    testset = CIFAR10(root=root_path, train=False, download=True, transform=normalize_transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    # see output on example from test set
    dataiter = iter(testloader)
    images, labels = next(dataiter)
    # print images
    # tensor_show(torchvision.utils.make_grid(images), is_normalized=True, one_channel=False, is_show=True)
    print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))
    # print predictions
    outputs = model(images)
    _, predicted = torch.max(outputs, dim=1)
    print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                                  for j in range(4)))

    # prediction on entire test set:
    # correct = 0
    # total = 0
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for (images, labels) in testloader:
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs, 1)

            # total += labels.size(0)
            # correct += (predicted == labels).sum().item()

            # collect the correct predictions for each class
            for label, prediction in zip(labels, predicted):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

    # print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
    print(f'Accuracy of the network on the overall test images: '
          f'{100 * sum(correct_pred.values()) // sum(total_pred.values())} %')

if __name__ == "__main__":
    main(sys.argv[1:])